# 3.2 - Reduccion de dimensiones

### PCA  (análisis de componente principal)

**PCA** es una transformación lineal usada para reducir dimensiones en los datos.

¿Por qué reducir dimensiones?

Existen varias razones, entre ellas:
+ Mejora de la performance
+ Mejor manejo de la dispersión de los datos
+ Maldición de las dimensiones (ojo-también existe la bendición)
+ etc...


Hay dos maneras diferentes de hacer ésta transformación:

+ A través de la matriz de correlaciones (dimensiones no homogéneas)
+ A través de la matriz de covarianzas (dimensiones homogéneas)

Ambas matrices son simétricas y diagonalizables. De hecho, el Teorema Espectral dice que si una matriz es hermítica, cuadrada y de dimensión finita, entonces existe una base de vectores propios donde dicha matriz puede ser representada.
Esto quiere decir que podemos cambiar de base para después proyectar, reduciendo las dimensiones e intentando conservar la máxima información en el nuevo subespacio.

![gio1](images/gioconda.jpeg)
![gio2](images/gioconda_lego.png)

#### Ejemplo intuitivo

In [ ]:
import pylab as plt
%matplotlib inline

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter([i for i in range(15)],
            [i+1 if i%2==0 else i-1 for i in range(15)])

plt.quiver(7, 7, 9, 4, color='r', scale=20)
plt.quiver(7, 7, -9, -4, color='r', scale=20)
plt.plot(7, 8, marker='$PC1$', ms=30, color='r')

plt.quiver(9, 9, -5, 4, color='b', scale=40)
plt.quiver(9, 9, 5, -4, color='b', scale=40)
plt.plot(8, 12, marker='$PC2$', ms=30, color='b');

Se rota y se proyecta, resultando:

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter([i for i in range(15)],
            [6 for i in range(15)])

plt.quiver(7, 6, 7, 0, color='r', scale=20)
plt.quiver(7, 6, -7, 0, color='r', scale=20)
plt.plot(7, 6.25, marker='$PC1$', ms=30, color='r');

##### Resumen PCA

+ Normalización de los datos
+ Obtener base de vectores propios desde matriz de correlacion o covarianza
+ Ordenar los vectores propios de mayor a menor según sus dimensiones en el nuevo subespacio
+ Matriz de proyección, con los autovectores seleccionados (W)
+ Se transforma X (los datos) según W (matriz de proyección)

In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler  # normalizacion

from sklearn.decomposition import PCA    # PCA

import warnings
warnings.simplefilter('ignore')

In [ ]:
data=pd.read_csv('../data/pulsar_stars.csv')

data.head()

In [ ]:
data=data.drop(columns=['target_class'])

In [ ]:
data.shape

#### normalización

Recordemos, el primer paso de PCA es la normalización de los datos. 

Primero, veamos la **estandarización**  ($N(\mu, \sigma)$):

$$\frac{x-\mu}{\frac{\sigma}{\sqrt{n}}}$$

In [ ]:
data_n_mano = (data - data.mean()) / data.std()

data_n = StandardScaler().fit_transform(data)

np.sum(data_n_mano - data_n)

Ahora el **MinMax** :

$$\frac{x-min}{max-min}$$

In [ ]:
data_mm_mano=(data - np.min(data))/(np.max(data) - np.min(data))

data_mm=MinMaxScaler().fit_transform(data)

np.sum(data_mm_mano - data_mm)

Se usa la standarización:

In [ ]:
data = StandardScaler().fit_transform(data)

Se aplica **PCA**

In [ ]:
pca = PCA()

pca.fit(data)

print(data.shape)

pca.explained_variance_ratio_

In [ ]:
plt.figure(figsize=(10, 5))

plt.plot(np.cumsum(pca.explained_variance_ratio_))

plt.xlabel('Numero de componentes')
plt.ylabel('% varianza')
plt.ylim([0, 1.01]);

In [ ]:
pca = PCA(n_components=4)


data_pca = pca.fit_transform(data)


df = pd.DataFrame(data_pca)

df.head()

In [ ]:
df.shape

In [ ]:
sum(pca.explained_variance_ratio_)

In [ ]:
pd.DataFrame(data).head()

In [ ]:
pd.DataFrame(pca.inverse_transform(df)).head()

In [ ]:
data=pd.read_csv('../data/pulsar_stars.csv')

X = data.drop('target_class', axis=1)

y = data.target_class

In [ ]:
from sklearn.ensemble import RandomForestClassifier as RFC

from sklearn.model_selection import train_test_split as tts

from sklearn.metrics import f1_score as f1

In [ ]:
%%time

# SIN PCA

X_norm = StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test = tts(X_norm, y, train_size=0.8)

rfc=RFC()

rfc.fit(X_train, y_train)

y_pred = rfc.predict(X_test)

print(X_norm.shape)

f1(y_test, y_pred)

In [ ]:
%%time

# CON PCA

X_norm = StandardScaler().fit_transform(X)

pca = PCA(n_components=4)

X_norm = pca.fit_transform(X_norm)

X_train, X_test, y_train, y_test = tts(X_norm, y, train_size=0.8)

rfc=RFC()

rfc.fit(X_train, y_train)

y_pred = rfc.predict(X_test)

print(X_norm.shape)

f1(y_test, y_pred)

### UMAP (uniform manifold aprox and projection)

Tiene dos pasos:

+ KNN con pesos, según topología (grafo)
+ Se reduce la dimensión basándose en esa topología

https://umap-learn.readthedocs.io/en/latest/

In [ ]:
%pip install umap-learn

In [ ]:
import umap

import seaborn as sns

from sklearn.datasets import load_iris

In [ ]:
load_iris().data.shape

In [ ]:
data=load_iris().data

In [ ]:
reduc=umap.UMAP(n_components=2).fit_transform(data)

reduc.shape

In [ ]:
load_iris().target

In [ ]:
import pylab as plt

plt.scatter(reduc[:,0], # x
            reduc[:, 1],# y 
            c=[sns.color_palette()[x] for x in load_iris().target])


plt.gca().set_aspect('equal', 'datalim')
plt.title('Proyeccion UMAP');

In [ ]:
%%time

# SIN UMAP

X_norm = StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test = tts(X_norm, y, train_size=0.8)

rfc=RFC()

rfc.fit(X_train, y_train)

y_pred = rfc.predict(X_test)

print(X_norm.shape)

f1(y_test, y_pred)

In [ ]:
%%time

# CON UMAP

X_norm = StandardScaler().fit_transform(X)

X_norm = umap.UMAP(n_components=4).fit_transform(X_norm)

X_train, X_test, y_train, y_test = tts(X_norm, y, train_size=0.8)

rfc=RFC()

rfc.fit(X_train, y_train)

y_pred = rfc.predict(X_test)

print(X_norm.shape)

f1(y_test, y_pred)

### t-SNE

**t-Distributed Stochastic Neighbor Embbeding**

Convierte similitudes entre los datos en probabilidad conjunta y trata de minimizar la divergencia _Kullback-Leibler_ (entropía relativa):

$$D_{KL}(P|Q)=\sum P(x)log(\frac{P(x)}{Q(x)})$$

https://scikit-learn.org/stable/auto_examples/manifold/plot_t_sne_perplexity.html

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
tsne = TSNE(n_components=2, perplexity=10)

emb = tsne.fit_transform(load_iris().data)

emb = pd.DataFrame(emb, columns=['e1', 'e2'])

emb.head()

In [ ]:
emb.plot.scatter(x='e1', 
                 y='e2', 
                 c=[sns.color_palette()[x] for x in load_iris().target])

plt.title('Proyeccion t-SNE');

In [ ]:
%%time

# SIN TSNE

X_norm = StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test = tts(X_norm, y, train_size=0.8)

rfc=RFC()

rfc.fit(X_train, y_train)

y_pred = rfc.predict(X_test)

print(X_norm.shape)

f1(y_test, y_pred)

In [ ]:
%%time

# CON TSNE

X_norm = StandardScaler().fit_transform(X)

X_norm = TSNE(n_components=2, perplexity=10).fit_transform(X_norm)

X_train, X_test, y_train, y_test = tts(X_norm, y, train_size=0.8)

rfc=RFC()

rfc.fit(X_train, y_train)

y_pred = rfc.predict(X_test)

print(X_norm.shape)

f1(y_test, y_pred)

## Guardar modelo de machine learning

In [ ]:
rfc.predict(X_test)[:10]

In [ ]:
rfc

In [ ]:
# guardar modelo ML

import pickle

pickle.dump(rfc, open('random_forest_pulsar.pk', 'wb')) # escribe en binario

In [ ]:
# cargar modelo

modelo_rf = pickle.load(open('random_forest_pulsar.pk', 'rb')) # lee en binario

In [ ]:
modelo_rf

In [ ]:
y_pred = modelo_rf.predict(X_test)

f1(y_test, y_pred)

In [ ]:
pickle.dump(pca, open('pca_pulsar.pk', 'wb')) # escribe en binario

In [ ]:
pickle.load(open('pca_pulsar.pk', 'rb')) # lee en binario

In [ ]:
modelo_rf

In [ ]:
# reentrenado desde el actual 

modelo_rf.fit(X_test, y_test)

y_pred = modelo_rf.predict(X_test)

f1(y_test, y_pred)